In [1]:
with open('测试方法和流程拆分出的内容.html', 'r', encoding = 'utf-8') as f:
    html = f.read()

In [2]:
from lxml import etree

In [3]:
import unittest

In [4]:
html = etree.HTML(html)

结构：(主题,内容)
如果没有内容，主题的图片作为内容
如果内容中有图片，图片作为内容的一部分

In [5]:
class qa():
    def __init__(self,obj):
        self.obj = obj
        self.pre = ''
        if not self.have_children() and not self.have_title_img():
            return False
        
    def get_obj(self):
        return self.obj
    
    def get_pre(self, node):
        data = node.xpath('../../../span');
        if data:
            return data.xpath('./@color')
        return False
    
    def make_pre(self):
        this_node = self.obj
        while this_node.xpath('../../../span/@color') == ["#3da8f5"]:
            if self.pre == '':
                tg =''
            else:
                tg = '/'
            self.pre = this_node.xpath('../../../span/text()')[0]+tg+self.pre
            this_node = this_node.xpath('../../../span')[0]
        if self.pre != '':
            return('[{}]'.format(self.pre))
        return ''
    
    def img_handle(self, img):
        if img:
            return [img[0].replace('https','http')][0]
        return []
#     img[0].replace('https','http')
    
    def have_children(self):
        return len(self.obj.xpath('../ul'))
    
    def have_title_img(self):
        return len(self.get_title_img())
    
    def get_img(self, obj):
        return self.img_handle(obj.xpath('../div/img/@src'))
    
    def get_title_img(self):
        return self.get_img(self.obj)
    
    def title(self):
        if self.have_children():
            if self.get_title_img():
                return '<p>{}</p><img src={} />'.format(self.obj.xpath('./text()')[0].replace(' ',''), self.get_title_img()[0])
            else:
                return '<p>{}</p>'.format(self.obj.xpath('./text()')[0].replace(' ',''))
        my_data = self.obj.xpath('./text()')
        return '<p>{}</p>'.format(my_data[0].replace(' ',''))
    def tag(self):
        return self.obj
    def detail(self):
        if not self.have_children():
            return '<img src={} />'.format(self.get_title_img())
        img = self.get_title_img()
        detail_text = self.obj.xpath('../ul/li/span/text()')
        if img:
            return('<p>{}</p><img src={} />'.format(detail_text, img))
        else:
            text = '</p><p>'.join(detail_text)
            return('<p>{}</p>'.format(text))

获取标题作为标记

In [6]:
tag = '{}知识点'.format(html.xpath('//title/text()')[0].replace(' - 幕布',""))
print(tag)

测试方法和流程拆分出的内容 知识点


In [7]:
datas = html.xpath('//span[@color="#3da8f5"]')
print(datas)

[<Element span at 0x216268fa288>, <Element span at 0x216268fa648>, <Element span at 0x216268fc748>, <Element span at 0x216268fcfc8>, <Element span at 0x216268fccc8>, <Element span at 0x216268fcf88>, <Element span at 0x216268e5b08>, <Element span at 0x216268e53c8>, <Element span at 0x216268e5ac8>, <Element span at 0x216268fc8c8>, <Element span at 0x216268e5cc8>, <Element span at 0x216268e5bc8>, <Element span at 0x216268e5a08>, <Element span at 0x216268e5d48>, <Element span at 0x216268e5248>, <Element span at 0x216268e5c88>, <Element span at 0x216268e5c48>, <Element span at 0x216268e5388>, <Element span at 0x21626901888>, <Element span at 0x21626901848>, <Element span at 0x21626901788>, <Element span at 0x21626901dc8>, <Element span at 0x21626901548>, <Element span at 0x21626901188>]


In [8]:
out_datas = [(qa(data).title(), qa(data).detail(),qa(data).make_pre()) for data in datas]

In [9]:
str = ''

In [10]:
# tag = 'concurrent.futures知识点'

In [11]:
for i in out_datas:
    str += '<p>[{}]{}</p> {}\t[{}]{} {}\n'.format(tag,i[2], i[0],tag,i[2],i[1])

In [12]:
print(str)

<p>[测试方法和流程拆分出的内容 知识点]</p> <p>因果图法说明</p>	[测试方法和流程拆分出的内容 知识点] <p>1、因：输入条件</p><p>2、果：输出结果</p><p>3、以画图的方式来表示输入条件和输出结果之间的关系</p>
<p>[测试方法和流程拆分出的内容 知识点]</p> <p>因果图法中的图形符号（掌握）</p>	[测试方法和流程拆分出的内容 知识点] <p>1、基本图形符号</p><p>2、限制图形符号</p>
<p>[测试方法和流程拆分出的内容 知识点]</p> <p>因果图法中的图形符号（掌握）--基本图形符号</p>	[测试方法和流程拆分出的内容 知识点] <p>表示因与果之间的关系（因—果）</p><p>a、恒等</p><p>b、非</p><p>c、与</p><p>d、或</p>
<p>[测试方法和流程拆分出的内容 知识点]</p> <p>因果图法中的图形符号（掌握）detail限制图形符号</p>	[测试方法和流程拆分出的内容 知识点] <p>表示要么因之间，要么果之间的限制关系（单方面）</p><p>1 互斥（E-exclude）：只能有一个或者没有</p><p>2 唯一（O-only）：有且只能有一个</p><p>3、包含（I-include）必须一个或者多个</p><p>4、要求（R-required）a的值决定b的值，反之b的值对a没有约束</p><p>5、屏蔽（M-masked）a如果a的值为1，b的值为0，a的值为0，b的值可以是1,也可以是0(if(a){b=0;return(b)}else{returnb})</p>
<p>[测试方法和流程拆分出的内容 知识点]</p> <p>判定表的特点3个无关紧要，1个互补影响</p>	[测试方法和流程拆分出的内容 知识点] <p>输入条件的顺序无关紧要</p><p>输出结果的顺序无关紧要</p><p>先测哪个组合（列），后测哪个组合无关紧要</p><p>每个组合都是相互独立的，互不影响。</p>
<p>[测试方法和流程拆分出的内容 知识点]</p> <p>判定表的组成部分</p>	[测试方法和流程拆分出的内容 知识点] <p>1、条件桩</p><p>2、动作桩</p><p>3、条件项</p><p>4、动作项</p>
<p>[测试方法和流程拆分出的内容 知识点]</p> 

In [13]:
with open('./anki_test.txt', 'w', encoding = 'utf-8') as f:
    f.write(str)